# Sample Workflow for d3rlpy Experiments

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler_confounder import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [5]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    samples.setting("coarse")
    states = []
    actions = []
    rewards = []
    metrics = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk, metricChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
        metrics.append(metricChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    metrics = np.hstack(metrics)
    
    terminals = np.zeros(len(states))
    terminals[::1111] = 1 #episode length 100, change if necessary
    dataset = d3rlpy.dataset.MDPDataset(np.hstack([states.numpy(), metrics[:,None]]), 
                                        actions.numpy(), 
                                        rewards.numpy(), 
                                        metrics, terminals)
    return dataset, states.numpy(), actions.numpy(), rewards.numpy(), metrics

We can build the dataset from there, just like this, and split into train and test sets.

In [6]:
dataset, statesOrig, actions, rewards, metrics = get_dataset([i for i in range(1000)], path="collected_data/rl_stochpid.txt")

start
[ 0.00000000e+00  7.95731469e+08 -4.75891077e-02 -3.69999953e-02
  2.00999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.50429671e-01 -4.92727243e-01 -5.31666025e-03 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.25610892e-01 -3.35999953e-02
 -2.42000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08749986e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.90489108e-01 -5.87999953e-02
 -1.01000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.76979602e-02 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+0

[ 0.00000000e+00  7.95731469e+08  1.14110892e-01  5.68000047e-02
  7.99998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.54213589e-03  3.91473614e-01 -9.17637410e-02 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 48 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.60389108e-01  1.12000047e-02
  1.94999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.68637874e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 49 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.59710892e-01 -1.19999531e-03
  1.38999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -1.86128266e-01  3.12352813e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0

[ 0.00000000e+00  7.95731469e+08 -7.15891077e-02 -7.99999531e-03
  2.33999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.79765872e-01 -3.17402568e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 97 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.09789108e-01 -4.57999953e-02
 -2.20000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.84885686e-01 -2.78683294e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 98 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.58389108e-01  5.40000469e-03
 -2.15000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -4.93350106e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0

[ 0.00000000e+00  7.95731469e+08 -3.01891077e-02  5.08000047e-02
 -7.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.53141120e-01 -8.63096507e-02  2.37374998e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 145 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.02189108e-01  5.78000047e-02
  5.39998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.40938487e-01 -2.83568202e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 146 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.14710892e-01  4.44000047e-02
  1.91999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  1.85963953e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

[ 0.00000000e+00  7.95731469e+08 -3.40389108e-01  5.44000047e-02
  8.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.85300440e-02 -4.75342936e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 193 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -8.29891077e-02 -5.09999953e-02
  1.86999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.55508053e-01 -4.43781082e-01  4.05012232e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 194 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.13110892e-01  5.96000047e-02
 -2.41000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

...

[ 0.00000000e+00  7.95731469e+08  5.41108923e-02 -5.35999953e-02
  2.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -8.09880485e-02  2.50924100e-01 -4.52974371e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 779 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -1.74789108e-01 -1.67999953e-02
 -1.34000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.60230695e-01 -2.30482844e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 780 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.59889108e-01  1.12000047e-02
 -2.61000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

[ 0.00000000e+00  7.95731469e+08  4.24610892e-01 -4.09999953e-02
 -2.74000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 824 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.42989108e-01  3.84000047e-02
 -1.26000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.97331372e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 825 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.81710892e-01  4.84000047e-02
  2.88999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.74754239e-02  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

start
[ 0.00000000e+00  7.95731469e+08  3.84710892e-01 -4.93999953e-02
 -2.78000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.05778192e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 870 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -6.04891077e-02 -9.99995308e-04
 -9.60001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.03183804e-01 -1.95233071e-02  1.11702868e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 871 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.33010892e-01 -9.19999531e-03
 -6.20001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.61482841e-02  2.96465732e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.7781450

[ 0.00000000e+00  7.95731469e+08  1.42010892e-01 -2.87999953e-02
 -2.29000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 911 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.47410892e-01  4.02000047e-02
 -3.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.89646077e-02  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 912 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.37610892e-01 -3.33999953e-02
  4.69998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.53570111e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

start
[ 0.00000000e+00  7.95731469e+08 -2.76489108e-01 -5.51999953e-02
 -3.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.57069552e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 956 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.08289108e-01  3.56000047e-02
  1.78999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.41320942e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 957 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.15910892e-01 -2.87999953e-02
 -2.54000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.82318925e-01  4.73669145e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.7781450

In [7]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -128.34366,
 'std': 83.63828,
 'min': -410.7058,
 'max': 0.0,
 'histogram': (array([  4,   2,  11,  11,  22,  18,  25,  33,  39,  22,  22,  41,  73,
          76, 103, 118, 146, 179,  54,   1]),
  array([-410.7058  , -390.17053 , -369.63522 , -349.09995 , -328.56464 ,
         -308.02936 , -287.49408 , -266.95877 , -246.4235  , -225.8882  ,
         -205.3529  , -184.81761 , -164.28232 , -143.74704 , -123.21175 ,
         -102.67645 ,  -82.14116 ,  -61.605873,  -41.07058 ,  -20.53529 ,
            0.      ], dtype=float32))}

In [9]:
# plt.plot(states[:,6])

In [10]:
# plt.plot(actions)

In [11]:
#plt.plot(model.predict(np.array(states)))

In [12]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=5)

## Setting up an Algorithm

In [13]:
from d3rlpy.algos import CQL
from d3rlpy.models.encoders import VectorEncoderFactory

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

#actor_encoder = VectorEncoderFactory(hidden_units=[12, 24, 36, 24, 12],
#                                       activation='relu', use_batch_norm=True, dropout_rate=0.2)
#critic_encoder = VectorEncoderFactory(hidden_units=[12, 24, 24, 12],
#                                       activation='relu', use_batch_norm=True, dropout_rate=0.2)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=True) #change it to true if you have one
model.build_with_dataset(dataset)

In [14]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.11205020533958533


In [15]:
%load_ext tensorboard
%tensorboard --logdir runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/dasc/anaconda3/envs/jbreeden3.10/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/main.py", line 40, in <module>
    from tensorboard import default
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/default.py", line 38, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard import plugin_util
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 21, in <module>
    from tensorboard._vendor import bleach
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorb

In [16]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=20, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-17 14:30.51 [debug    ] RoundIterator is selected.
2022-04-17 14:30.51 [info     ] Directory is created at d3rlpy_logs/CQL_20220417143051
2022-04-17 14:30.51 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-17 14:30.51 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-17 14:30.51 [warning  ] Skip building models since they're already built.
2022-04-17 14:30.51 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220417143051/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/20:   0%|          | 0/4309 [00:00<?, ?it/s]

2022-04-17 14:31.38 [info     ] CQL_20220417143051: epoch=1 step=4309 epoch=1 metrics={'time_sample_batch': 0.00025647398655861096, 'time_algorithm_update': 0.010486876391899727, 'temp_loss': 3.394779772901236, 'temp': 0.8271505036329719, 'alpha_loss': -5.329435448875717, 'alpha': 1.1590072212044393, 'critic_loss': 11.456173180039894, 'actor_loss': 13.556259882130398, 'time_step': 0.01089877417204302, 'td_error': 14.039907260250546, 'init_value': -36.51660919189453, 'ave_value': -23.69267963622168} step=4309
2022-04-17 14:31.38 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417143051/model_4309.pt


Epoch 2/20:   0%|          | 0/4309 [00:00<?, ?it/s]

2022-04-17 14:32.26 [info     ] CQL_20220417143051: epoch=2 step=8618 epoch=2 metrics={'time_sample_batch': 0.00026100028775689443, 'time_algorithm_update': 0.010442913809683572, 'temp_loss': 1.5059043224905284, 'temp': 0.5785030979143547, 'alpha_loss': 1.0276504781109979, 'alpha': 1.1825857058316158, 'critic_loss': 36.939313554907656, 'actor_loss': 62.00821614304173, 'time_step': 0.010876100614967797, 'td_error': 23.539338564792306, 'init_value': -85.83417510986328, 'ave_value': -64.78024209718849} step=8618
2022-04-17 14:32.26 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417143051/model_8618.pt


...

Epoch 19/20:   0%|          | 0/4309 [00:00<?, ?it/s]

2022-04-17 14:48.05 [info     ] CQL_20220417143051: epoch=19 step=81871 epoch=19 metrics={'time_sample_batch': 0.0003489413873367504, 'time_algorithm_update': 0.01612538377324067, 'temp_loss': -0.0010165284560620699, 'temp': 0.14930280870491072, 'alpha_loss': -0.010404387441515948, 'alpha': 0.34648079730623915, 'critic_loss': 23.361932005841318, 'actor_loss': 43.79587581932863, 'time_step': 0.016744195239177074, 'td_error': 2.862667779896601, 'init_value': -55.07027053833008, 'ave_value': -22.754897393433897} step=81871
2022-04-17 14:48.05 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417143051/model_81871.pt


Epoch 20/20:   0%|          | 0/4309 [00:00<?, ?it/s]

2022-04-17 14:49.17 [info     ] CQL_20220417143051: epoch=20 step=86180 epoch=20 metrics={'time_sample_batch': 0.0003393909775704512, 'time_algorithm_update': 0.015996473243325624, 'temp_loss': -0.0012708520888504143, 'temp': 0.1519829111299539, 'alpha_loss': 0.004844973477449226, 'alpha': 0.3466879540099487, 'critic_loss': 23.24989951414917, 'actor_loss': 38.84604859944147, 'time_step': 0.01660438595362566, 'td_error': 2.666530147944117, 'init_value': -48.76219940185547, 'ave_value': -17.776884348079697} step=86180
2022-04-17 14:49.17 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220417143051/model_86180.pt


[(1,
  {'time_sample_batch': 0.00025647398655861096,
   'time_algorithm_update': 0.010486876391899727,
   'temp_loss': 3.394779772901236,
   'temp': 0.8271505036329719,
   'alpha_loss': -5.329435448875717,
   'alpha': 1.1590072212044393,
   'critic_loss': 11.456173180039894,
   'actor_loss': 13.556259882130398,
   'time_step': 0.01089877417204302,
   'td_error': 14.039907260250546,
   'init_value': -36.51660919189453,
   'ave_value': -23.69267963622168}),
 (2,
  {'time_sample_batch': 0.00026100028775689443,
   'time_algorithm_update': 0.010442913809683572,
   'temp_loss': 1.5059043224905284,
   'temp': 0.5785030979143547,
   'alpha_loss': 1.0276504781109979,
   'alpha': 1.1825857058316158,
   'critic_loss': 36.939313554907656,
   'actor_loss': 62.00821614304173,
   'time_step': 0.010876100614967797,
   'td_error': 23.539338564792306,
   'init_value': -85.83417510986328,
   'ave_value': -64.78024209718849}),
 (3,
  {'time_sample_batch': 0.0002515944562563627,
   'time_algorithm_update':

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [17]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset, states_ope, actions_ope, rewards_ope, metrics_ope = get_dataset([500+i for i in range(50)], path="collected_data/rl_stochpid.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=5)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes,
        eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

start
[ 0.00000000e+00  7.95731469e+08 -4.19889108e-01 -1.83999953e-02
  2.71999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 501 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.30889108e-01 -2.27999953e-02
 -3.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.19275575e-02 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 502 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.99289108e-01 -3.69999953e-02
  2.18999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.02595006e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.7781450

[ 0.00000000e+00  7.95731469e+08  1.26710892e-01  7.60000469e-03
 -1.01000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  4.54273830e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 531 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  7.57108923e-02  5.62000047e-02
  8.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.86939797e-02  1.54513207e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 532 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.60210892e-01 -1.49999953e-02
  4.19998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.25177022e-02  3.21207732e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

Epoch 1/50:   0%|          | 0/499 [00:00<?, ?it/s]

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2022-04-17 14:49.19 [info     ] FQE_20220417144917: epoch=1 step=499 epoch=1 metrics={'time_sample_batch': 0.00017088209698816578, 'time_algorithm_update': 0.00361565979783664, 'loss': 0.0064157983843416335, 'time_step': 0.0038696974217294453, 'init_value': -0.5316524505615234, 'ave_value': -0.3708868765649763, 'soft_opc': nan} step=499


...

2022-04-17 14:51.01 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220417144917/model_24451.pt


Epoch 50/50:   0%|          | 0/499 [00:00<?, ?it/s]

2022-04-17 14:51.03 [info     ] FQE_20220417144917: epoch=50 step=24950 epoch=50 metrics={'time_sample_batch': 0.0001380429239215736, 'time_algorithm_update': 0.003499410912125765, 'loss': 0.05065426302682758, 'time_step': 0.003701590344996634, 'init_value': -6.726969242095947, 'ave_value': -4.322503234024856, 'soft_opc': nan} step=24950


2022-04-17 14:51.03 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220417144917/model_24950.pt


[(1,
  {'time_sample_batch': 0.00017088209698816578,
   'time_algorithm_update': 0.00361565979783664,
   'loss': 0.0064157983843416335,
   'time_step': 0.0038696974217294453,
   'init_value': -0.5316524505615234,
   'ave_value': -0.3708868765649763,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.0001712734092452483,
   'time_algorithm_update': 0.003700443164619033,
   'loss': 0.004909136375498198,
   'time_step': 0.003952516582542526,
   'init_value': -0.606988251209259,
   'ave_value': -0.39794752515919574,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00017068285741404684,
   'time_algorithm_update': 0.0037714713561033198,
   'loss': 0.0040915756946418415,
   'time_step': 0.004023511806327499,
   'init_value': -0.644119918346405,
   'ave_value': -0.4146075309094813,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00015766396073396794,
   'time_algorithm_update': 0.0034528452313256886,
   'loss': 0.0036742373671520777,
   'time_step': 0.0036867032786887253,
   'init_

In [18]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset, states_ope, actions_ope, rewards_ope, metrics_ope = get_dataset([2,4,6,8], path="collected_data/rl_stoch_small.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [19]:
model.save_policy("workflowConfound.pt")
model.save_model("workflowConfoundmodel.pt")

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


In [21]:
from d3rlpy.torch_utility import to_cpu
to_cpu(model)
model.save_policy("workflowConfoundCPU.pt")
model.save_model("workflowConfoundmodelCPU.pt")